This workbook add DPO related files into 3 vector store in order to check:
1. How irrelevant files impact the retriver performance
2. How chunk size impact retriver performance

In [ ]:
# https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-qa/question_answering_documents_langchain_matching_engine.ipynb

In [10]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/internal_knowledge_chatbot/src/utils')

In [11]:
from doc_to_vertex_search import *

# 2. Load pdfs

In [56]:
from loader_splitter import *

##### - load cnils only files

In [19]:
!ls ../documents/ecg_dpo/

 CNIL	      'Code de commerce'	 'Code du tourisme'
'Code civil'  'Code de la consommation'  'Code du travail'


In [76]:
cnil_docs, unloaded_pdfs, loaded_pdfs, other_files = process_folder(folder_path=f"../documents/ecg_dpo/CNIL",unstructure=True)

[2024-05-15 10:16:23] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf
Using unstrucure loader


/home/qianyucazelles/internal_knowledge_chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-15 10:16:27] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE I - Dispositions générales.pdf
Using unstrucure loader
[2024-05-15 10:16:28] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispositions finales _ CNIL.pdf
Using unstrucure loader
[2024-05-15 10:16:28] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE V - Transferts de données à caractère personnel vers des pays tiers ou à des organisations internationales _ CNIL.pdf
Using unstrucure loader
[2024-05-15 10:16:30] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VII - Coopération et cohérence _ CNIL.pdf
Using unstrucure loader
[2024-05-15 10:16:33] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VI - Autorités de contrôle indépendantes _ CNIL.pdf
Using unstrucure loader
[2024-05-15 10:16:35] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE III - Droits de la personne concernée _ CNIL.pdf
Using unstrucure loader
[2024-05-15 10:16:37] Scanning ../documents/ecg_dpo/CNIL/CHAPITRE VIII - Voies de recours, responsabilité et sanctions _ CNIL.pdf
U

In [77]:
len(unloaded_pdfs)

0

In [78]:
len(cnil_docs)

11

##### - load all dpo files

In [41]:
!ls ../documents/ecg_assurance/'Contrats 2024'/CAMPINGS/ALLIANZ

'Avenant 2024 Police campings Allianz.pdf'
'Avenant LPS 2024 Police campings Allianz.pdf'
'Avenant police campings Allianz 2024.pdf'
'Avenant police campings LPS Allianz 2024.pdf'


In [224]:
def process_folder(folder_path, unstructure=False):
    docs = []
    loaded_pdfs = []  
    unloaded_pdfs = []
    other_files = []
    
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        print_with_time(f"Scanning {item_path}")
        
        if os.path.isdir(item_path):
            print_with_time(f"{item_path} is folder, now go to new level")
            # Recursively process the subfolder and capture the returned values
            sub_docs, sub_unloaded_pdfs, sub_loaded_pdfs, sub_other_files = process_folder(item_path, unstructure)
            docs.extend(sub_docs)
            unloaded_pdfs.extend(sub_unloaded_pdfs)
            loaded_pdfs.extend(sub_loaded_pdfs)
            other_files.extend(sub_other_files)
            
        elif item.lower().endswith('.pdf'):
            try:
                doc = load_pdf_to_langchain_doc(item_path, unstructure_loader=unstructure)
                if len(doc[0].page_content) > 10:
                    docs += doc
                    loaded_pdfs.append(item_path)    
                else:
                    unloaded_pdfs.append(item_path)    
            except Exception as e:
                unloaded_pdfs.append(item_path)     
        else:
            other_files.append(item_path)
            
    return docs, unloaded_pdfs, loaded_pdfs, other_files

In [226]:
dpo_docs, unloaded_pdfs, loaded_pdfs, other_files = process_folder(folder_path=f"../documents/ecg_dpo",unstructure=True)

[2024-05-15 13:38:49] Scanning ../documents/ecg_dpo/Code civil
[2024-05-15 13:38:49] ../documents/ecg_dpo/Code civil is folder, now go to new level
[2024-05-15 13:38:49] Scanning ../documents/ecg_dpo/Code civil/LEGITEXT000006070721.pdf
Using unstrucure loader
[2024-05-15 13:39:27] Scanning ../documents/ecg_dpo/Code de la consommation
[2024-05-15 13:39:27] ../documents/ecg_dpo/Code de la consommation is folder, now go to new level
[2024-05-15 13:39:27] Scanning ../documents/ecg_dpo/Code de la consommation/LEGITEXT000006069565.pdf
Using unstrucure loader
[2024-05-15 13:40:14] Scanning ../documents/ecg_dpo/Code du travail
[2024-05-15 13:40:14] ../documents/ecg_dpo/Code du travail is folder, now go to new level
[2024-05-15 13:40:14] Scanning ../documents/ecg_dpo/Code du travail/LEGITEXT000006072050.pdf
Using unstrucure loader
[2024-05-15 13:44:06] Scanning ../documents/ecg_dpo/Code du tourisme
[2024-05-15 13:44:06] ../documents/ecg_dpo/Code du tourisme is folder, now go to new level
[2024-

In [227]:
len(dpo_docs)

16

In [229]:
len(unloaded_pdfs)

0

# 3. Split doc

##### - cnil_splits

In [80]:
# cnil_docs[0]

In [82]:
# cnil_docs[5].page_content

In [135]:
# split articles to chunk
from langchain.text_splitter import RecursiveCharacterTextSplitter

def doc_to_splits( docs,
                  separators = ["\n\n", ".", "!", "?"],
                  chunk_size= 1500,
                  chunk_overlap=150,         
                  ):

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators
    )

    # add chunk number
    doc_splits_list = []
    for doc in docs:
        doc_splits = text_splitter.split_documents([doc])

        # Add chunk number to metadata
        for idx, split in enumerate(doc_splits):
            split.metadata["chunk"] = idx+1
            split.metadata["total_chunks"] = len(doc_splits)
        doc_splits_list+=doc_splits

    return doc_splits_list

In [148]:
def get_docs_min_max_token_length(docs):
    min_nb = 1000
    min_idx = 0
    max_nb = 0
    max_idx = 0
    for _, doc in enumerate(docs):
        nb_ = nb_tokens(doc.page_content)
        if nb_>max_nb:
            max_nb = nb_
            max_idx = _
        if nb_<min_nb:
            min_nb = nb_
            min_idx = _
    return {min_idx:min_nb,max_idx:max_nb}

In [149]:
get_docs_min_max_token_length(cnil_docs)

{9: 421, 10: 9800}

##### split to large chunk size referring to langchain

In [163]:
cnil_large_splits = doc_to_splits(cnil_docs,
                  separators = ["\n\nArticle","\n\n", ".", "!", "?"],
                  chunk_size= 1500,
                  chunk_overlap=150,         
                  )

In [166]:
len(cnil_large_splits)

230

In [164]:
get_docs_min_max_token_length(cnil_large_splits)

{34: 15, 111: 271}

In [165]:
cnil_large_splits[34]

Document(page_content='https://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre11\n\n2/3\n\n4/15/24, 9:58 AM\n\nCHAPITRE XI - Dispositions finales | CNIL', metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE XI - Dispositions finales _ CNIL.pdf', 'chunk': 4, 'total_chunks': 5})

In [174]:
cnil_large_splits[210].metadata['source']

'../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf'

##### - split to small chunk sizes based on google vertex

In [176]:
cnil_small_splits = doc_to_splits(cnil_docs,
                  separators = ["\n\nArticle","\n\n", ".", "!", "?"],
                  chunk_size= 500,
                  chunk_overlap=50,         
                  )

In [178]:
len(cnil_small_splits)

703

In [177]:
get_docs_min_max_token_length(cnil_small_splits)

{17: 1, 115: 243}

In [179]:
cnil_small_splits[17]

Document(page_content='4', metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf', 'chunk': 18, 'total_chunks': 50})

In [180]:
cnil_small_splits[18]

Document(page_content=". sur le consentement de la personne concernée ou sur le droit de l'Union ou le droit d'un État membre qui constitue une mesure nécessaire et proportionnée dans une société démocratique pour garantir les objectifs visés à l'article 23, paragraphe 1, le responsable du traitement, afin de déterminer si le traitement à une autre fin est compatible avec la finalité pour laquelle les données à caractère personnel ont été initialement collectées, tient compte, entre autres: a) de l'existence éventuelle d'un lien entre les finalités pour lesquelles les données à caractère personnel ont été collectées et les finalités du traitement ultérieur envisagé; b) du contexte dans lequel les données à caractère personnel ont été collectées, en particulier en ce qui concerne la relation entre les personnes concernées et le responsable du traitement;", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE II - Principes _ CNIL.pdf', 'chunk': 19, 'total_chunks': 50})

##### - dpo splits

In [230]:
dpo_splits = doc_to_splits(dpo_docs,
                  separators = ["\n\nArticle","\n\n", ".", "!", "?"],
                  chunk_size= 1500,
                  chunk_overlap=150,         
                  )

In [231]:
dpo_splits[0]

Document(page_content="Code civil\n\nTitre préliminaire : De la publication, des effets et de l'application des lois en général\n\nArticle 1\n\nLes lois et, lorsqu'ils sont publiés au Journal officiel de la République française, les actes administratifs entrent en vigueur à la date qu'ils fixent ou, à défaut, le lendemain de leur publication. Toutefois, l'entrée en vigueur de celles de leurs dispositions dont l'exécution nécessite des mesures d'application est reportée à la date d'entrée en vigueur de ces mesures.\n\nEn cas d'urgence, entrent en vigueur dès leur publication les lois dont le décret de promulgation le prescrit et les actes administratifs pour lesquels le Gouvernement l'ordonne par une disposition spéciale.\n\nLes dispositions du présent article ne sont pas applicables aux actes individuels.\n\nArticle 2\n\nLa loi ne dispose que pour l'avenir ; elle n'a point d'effet rétroactif.\n\nArticle 3\n\nLes lois de police et de sûreté obligent tous ceux qui habitent le territoire.

# 4. Add to vector store

In [1]:
from langchain_google_vertexai import (
    VertexAIEmbeddings,
    VectorSearchVectorStore
)

In [2]:
# https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko-multilingual@001")

##### - cnil large 

In [182]:
parameters = get_me_parameters('../vector_store_me_parameters/dpo_cnil_me.json')

In [183]:
cnil_large_db = get_vector_store(parameters, embeddings)

In [ ]:
# Store docs as embeddings in Matching Engine index
# It may take a while since API is rate limited
texts = [doc.page_content for doc in cnil_large_splits]
metadatas = [doc.metadata for doc in cnil_large_splits]

for i in range(0, len(texts), 1000):
    cnil_large_db.add_texts(texts[i : i + 1000], metadatas[i : i + 1000])

##### - cnil small

In [187]:
!gsutil cp gs://vector_search_paramters/dpo_cnil_small_chunk_me.json ../vector_store_me_parameters/

Copying gs://vector_search_paramters/dpo_cnil_small_chunk_me.json...
/ [1 files][  423.0 B/  423.0 B]                                                
Operation completed over 1 objects/423.0 B.                                      


In [202]:
parameters = get_me_parameters('../vector_store_me_parameters/dpo_cnil_small_chunk_me.json')

In [204]:
parameters

{'PROJECT_ID': 'ecg-ai-416210',
 'LOCATION': 'europe-west1',
 'CHATBOT_NAME': 'dpo_cnil_small_chunk',
 'ME_INDEX_ID': 'projects/500033913879/locations/europe-west1/indexes/5755046169919094784',
 'ME_INDEX_ENDPOINT_ID': 'projects/500033913879/locations/europe-west1/indexEndpoints/5776367899404926976',
 'ME_INDEX_NAME': 'dpo_cnil_small_chunk-me-index',
 'ME_EMBEDDING_DIR': 'dpo_cnil_small_chunk/me-bucket',
 'ME_DIMENSIONS': 768}

In [206]:
cnil_small_db = get_vector_store(parameters, embeddings)

In [207]:
# Store docs as embeddings in Matching Engine index
# It may take a while since API is rate limited
texts = [doc.page_content for doc in cnil_small_splits]
metadatas = [doc.metadata for doc in cnil_small_splits]

for i in range(0, len(texts), 1000):
    cnil_small_db.add_texts(texts[i : i + 1000], metadatas[i : i + 1000])

Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/5755046169919094784
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/5755046169919094784


##### - dpo all

In [232]:
parameters = get_me_parameters('../vector_store_me_parameters/ecg_dpo_me.json')

In [235]:
dpo_db = get_vector_store(parameters, embeddings)

In [236]:
# Store docs as embeddings in Matching Engine index
# It may take a while since API is rate limited
texts = [doc.page_content for doc in dpo_splits]
metadatas = [doc.metadata for doc in dpo_splits]

for i in range(0, len(texts), 1000):
    dpo_db.add_texts(texts[i : i + 1000], metadatas[i : i + 1000])

Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
Upserting datapoints MatchingEngineIndex index: projects/500033913879/locations/europe-west1/indexes/1278468140312821760
MatchingEngineIndex index Upserted datapoints. Resource name: projects/500033913879/locations/europe-west1/inde

# 5. Similarity Search comparison

##### - Q1

In [193]:
q1 = "Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?"

Chapitre III page 2

In [196]:
cnil_large_db.similarity_search(q1, k=2, search_distance=0.4)

[Document(page_content="3. suite d'une demande formulée en application des articles 15 à 22, dans les meilleurs délais et en tout état de cause dans un délai d'un mois à compter de la réception de la demande. Au besoin, ce délai peut être prolongé de deux mois, compte tenu de la complexité et du nombre de demandes. Le responsable du traitement informe la personne concernée de cette prolongation et des motifs du report dans un délai d'un mois à compter de la réception de la demande. Lorsque la personne concernée présente sa demande sous une forme électronique, les informations sont fournies par voie électronique lorsque cela est possible, à moins que la personne concernée ne demande qu'il en soit autrement.\n\n4. Si le responsable du traitement ne donne pas suite à la demande formulée par la personne concernée, il informe celle-ci sans tarder et au plus tard dans un délai d'un mois à compter de la réception de la demande\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/ch

In [208]:
cnil_small_db.similarity_search(q1, k=2, search_distance=0.4)

[Document(page_content="4. Si le responsable du traitement ne donne pas suite à la demande formulée par la personne concernée, il informe celle-ci sans tarder et au plus tard dans un délai d'un mois à compter de la réception de la demande\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre3\n\n2/14\n\n4/15/24, 9:56 AM\n\nCHAPITRE III - Droits de la personne concernée | CNIL", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE III - Droits de la personne concernée _ CNIL.pdf', 'chunk': 12, 'total_chunks': 98}),
 Document(page_content="2. L'autorité de contrôle auprès de laquelle la réclamation a été introduite informe l'auteur de la réclamation de l'état d'avancement et de l'issue de la réclamation, y compris de la possibilité d'un recours juridictionnel en vertu de l'article 78.\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre8\n\n1/7\n\n4/15/24, 9:58 AM\n\nCHAPITRE VIII - Voies de recours, responsabilité et sanctions | CNIL", metadata=

In [237]:
dpo_db.similarity_search(q1, k=2, search_distance=0.4)

[Document(page_content="3. suite d'une demande formulée en application des articles 15 à 22, dans les meilleurs délais et en tout état de cause dans un délai d'un mois à compter de la réception de la demande. Au besoin, ce délai peut être prolongé de deux mois, compte tenu de la complexité et du nombre de demandes. Le responsable du traitement informe la personne concernée de cette prolongation et des motifs du report dans un délai d'un mois à compter de la réception de la demande. Lorsque la personne concernée présente sa demande sous une forme électronique, les informations sont fournies par voie électronique lorsque cela est possible, à moins que la personne concernée ne demande qu'il en soit autrement.\n\n4. Si le responsable du traitement ne donne pas suite à la demande formulée par la personne concernée, il informe celle-ci sans tarder et au plus tard dans un délai d'un mois à compter de la réception de la demande\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/ch

##### - Q2

In [197]:
q2 = "J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?"

IV page 3-7 ARTICLE 26-29

In [199]:
cnil_large_db.similarity_search(q2, k=6, search_distance=0.4)

[Document(page_content="4. Lorsqu'un sous-traitant recrute un autre sous-traitant pour mener des activités de traitement spécifiques pour le compte du responsable du traitement, les mêmes obligations en matière de protection de données que celles fixées dans le contrat ou un autre acte juridique entre le responsable du traitement et le sous-traitant conformément au paragraphe 3, sont imposées à cet autre sous-traitant par contrat ou au moyen d'un autre acte juridique au titre du droit de l'Union ou du droit d'un État membre, en particulier pour ce qui est de présenter des garanties suffisantes quant à la mise en œuvre de mesures techniques et organisationnelles appropriées de manière à ce que le traitement réponde aux exigences du présent règlement. Lorsque cet autre sous-traitant ne remplit pas ses obligations en matière de protection des données, le sous-traitant initial\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre4\n\n6/23\n\n4/15/24, 9:57 AM\n\nCHAPITRE 

In [210]:
cnil_small_db.similarity_search(q2, k=6, search_distance=0.4)

[Document(page_content="Article 28 - Sous-traitant\n\n1. uniquement appel à des sous-traitants qui présentent des garanties suffisantes quant à la mise en œuvre de mesures techniques et organisationnelles appropriées de manière à ce que le traitement réponde aux exigences du présent règlement et garantisse la protection des droits de la personne concernée. Le sous-traitant ne recrute pas un autre sous-traitant sans l'autorisation écrite préalable, spécifique ou", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf', 'chunk': 26, 'total_chunks': 177}),
 Document(page_content='. Lorsque cet autre sous-traitant ne remplit pas ses obligations en matière de protection des données, le sous-traitant initial', metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf', 'chunk': 39, 'total_chunks': 177}),
 Document(page_content=". Lorsqu'un sous-traitant recrute un autre sous-

In [238]:
dpo_db.similarity_search(q2, k=6, search_distance=0.4)

[Document(page_content="3° L'adresse des lieux successifs où doit s'accomplir la prestation, la date du début de la prestation et sa date de fin prévisible, l'activité principale exercée dans le cadre de la prestation ;\n\nCode du travail - Dernière modification le 05 avril 2024 - Document généré le 09 avril 2024\n\n4° Les nom, prénoms, date et lieu de naissance, adresse de résidence habituelle et nationalité de chacun des salariés détachés.\n\nSection 6 : Obligations et responsabilité financière des maîtres d'ouvrage et des donneurs d'ordre", metadata={'source': '../documents/ecg_dpo/Code du travail/LEGITEXT000006072050.pdf', 'chunk': 2847, 'total_chunks': 6703}),
 Document(page_content="Article R7343-106\n\nLe sous-traitant, dont l'implication dans une expertise ne peut concerner qu'une partie des travaux, agit sous l'autorité de l'expert.\n\nPour chaque expertise pour laquelle il recourt à un sous-traitant, l'expert s'assure que le sous-traitant remplit les conditions prévues aux 1°

##### - Q3

In [200]:
q3 = "Je ne sais pas quand est-ce qu'une analyse d'impact est indispensable ou non pour un traitement. Peux-tu m'expliquer clairement quand est-ce que je dois les réaliser et de manière simple ?"

IV ARTICLE 35, 12-13

In [201]:
cnil_large_db.similarity_search(q3, k=6, search_distance=0.4)

[Document(page_content="11. effectué conformément à l'analyse d'impact relative à la protection des données, au moins quand il se produit une modification du risque présenté par les opérations de traitement.\n\nSi nécessaire, le responsable du traitement procède à un examen afin d'évaluer si le traitement est\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre4\n\n13/23\n\n4/15/24, 9:57 AM\n\nCHAPITRE IV - Responsable du traitement et sous-traitant | CNIL", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf', 'chunk': 29, 'total_chunks': 55}),
 Document(page_content="3. dans les cas suivants:\n\nL'analyse d'impact relative à la protection des données visée au paragraphe 1 est, en particulier, requise\n\na) l'évaluation systématique et approfondie d'aspects personnels concernant des personnes physiques, qui est fondée sur un traitement automatisé, y compris le profilage, et sur la base de laquelle sont 

In [212]:
cnil_small_db.similarity_search(q3, k=6, search_distance=0.4)

[Document(page_content=". Une seule et même analyse peut porter sur un ensemble d'opérations de traitement similaires qui présentent des risques élevés similaires. Lorsqu'il effectue une analyse d'impact relative à la protection des données, le responsable du traitement", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf', 'chunk': 82, 'total_chunks': 177}),
 Document(page_content="11. effectué conformément à l'analyse d'impact relative à la protection des données, au moins quand il se produit une modification du risque présenté par les opérations de traitement.\n\nSi nécessaire, le responsable du traitement procède à un examen afin d'évaluer si le traitement est\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre4\n\n13/23\n\n4/15/24, 9:57 AM\n\nCHAPITRE IV - Responsable du traitement et sous-traitant | CNIL", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et s

In [239]:
dpo_db.similarity_search(q3, k=6, search_distance=0.4)

[Document(page_content="11. effectué conformément à l'analyse d'impact relative à la protection des données, au moins quand il se produit une modification du risque présenté par les opérations de traitement.\n\nSi nécessaire, le responsable du traitement procède à un examen afin d'évaluer si le traitement est\n\nhttps://www.cnil.fr/fr/reglement-europeen-protection-donnees/chapitre4\n\n13/23\n\n4/15/24, 9:57 AM\n\nCHAPITRE IV - Responsable du traitement et sous-traitant | CNIL", metadata={'source': '../documents/ecg_dpo/CNIL/CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf', 'chunk': 29, 'total_chunks': 55}),
 Document(page_content="3. dans les cas suivants:\n\nL'analyse d'impact relative à la protection des données visée au paragraphe 1 est, en particulier, requise\n\na) l'évaluation systématique et approfondie d'aspects personnels concernant des personnes physiques, qui est fondée sur un traitement automatisé, y compris le profilage, et sur la base de laquelle sont 

# 6. QA Chain test

In [240]:
# Create chain to answer questions
NUMBER_OF_RESULTS = 10
SEARCH_DISTANCE_THRESHOLD = 0.6

# Expose index to the retriever
retriever = cnil_large_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
    filters=None,
)

In [241]:
template = """SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: {question}

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

=============
{context}
=============

Question: {question}
Helpful Answer:"""

In [245]:
# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison@002",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
)


In [247]:
from langchain.prompts import PromptTemplate

In [248]:
# Uses LLM to synthesize results from the search index.
# Use Vertex PaLM Text API for LLM
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)

In [249]:
# Enable for troubleshooting
qa.combine_documents_chain.verbose = True
qa.combine_documents_chain.llm_chain.verbose = True
qa.combine_documents_chain.llm_chain.llm.verbose = True

In [252]:
import textwrap

In [250]:
def formatter(result):
    print(f"Query: {result['query']}")
    print("." * 80)
    if "source_documents" in result.keys():
        for idx, ref in enumerate(result["source_documents"]):
            print("-" * 80)
            print(f"REFERENCE #{idx}")
            print("-" * 80)
            if "score" in ref.metadata:
                print(f"Matching Score: {ref.metadata['score']}")
            if "source" in ref.metadata:
                print(f"Document Source: {ref.metadata['source']}")
            if "document_name" in ref.metadata:
                print(f"Document Name: {ref.metadata['document_name']}")
            print("." * 80)
            print(f"Content: \n{wrap(ref.page_content)}")
    print("." * 80)
    print(f"Response: {wrap(result['result'])}")
    print("." * 80)


def wrap(s):
    return "\n".join(textwrap.wrap(s, width=120, break_long_words=False))


def ask(
    query,
    qa=qa,
    k=NUMBER_OF_RESULTS,
    search_distance=SEARCH_DISTANCE_THRESHOLD,
    filters={},
):
    qa.retriever.search_kwargs["search_distance"] = search_distance
    qa.retriever.search_kwargs["k"] = k
    qa.retriever.search_kwargs["filters"] = filters
    result = qa({"query": query})
    return formatter(result)

In [253]:
ask(q1)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

3. suite d'une demande formulée en application des articles 15 à 22, dans les meilleurs délais et en tout état de cause dans un délai d'un mois à compter de la réception de la demande. Au besoin, ce délai peut être prolongé de deux mois, compte

In [256]:
ask(q2,k=2)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

4. Lorsqu'un sous-traitant recrute un autre sous-traitant pour mener des activités de traitement spécifiques pour le compte du responsable du traitement, les mêmes obligations en matière de protection de données que celles fixées dan

In [255]:
ask(q3,k=2)



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are an intelligent assistant helping the users with their questions on research papers.

Question: Je ne sais pas quand est-ce qu'une analyse d'impact est indispensable ou non pour un traitement. Peux-tu m'expliquer clairement quand est-ce que je dois les réaliser et de manière simple ?

Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

Do not try to make up an answer:
 - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
 - If the context is empty, just say "I do not know the answer to that."

11. effectué conformément à l'analyse d'impact relative à la protection des données, au moins quand il se produit une modification du risque présenté par les opérations de traitement.

Si nécessaire, le 